In [3]:
import pandas as pd
import yfinance as yf
from pandas_datareader import data
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import talib 
from dateutil.relativedelta import relativedelta

#from Tools import *
#from Tools2 import *
from Tool_v1 import *
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
yf.pdr_override()

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


In [4]:

hk_stocks_random=['0237.HK', '1771.HK', '1118.HK', '0570.HK', '3396.HK', '1515.HK', '2314.HK', '1069.HK', '1966.HK', '1013.HK', '0330.HK', '2948.HK', '8066.HK', '0046.HK', '0605.HK', '1766.HK', '0372.HK', '1300.HK', '1115.HK', '3363.HK', '2011.HK', '0242.HK', '0648.HK', '1808.HK', '1139.HK', '1383.HK', '0160.HK', '0820.HK', '8028.HK', '0366.HK', '0731.HK', '0802.HK', '1396.HK', '0604.HK', '1266.HK', '1189.HK', '0224.HK', '8193.HK', '0184.HK', '0585.HK', '0431.HK', '0883.HK', '0254.HK', '0455.HK', '1057.HK', '0156.HK', '0093.HK', '2607.HK', '3027.HK', '8162.HK', '0343.HK', '2009.HK', '0480.HK', '0730.HK', '0868.HK', '0687.HK', '6288.HK', '2362.HK', '0295.HK', '0023.HK', '0840.HK', '0590.HK', '0660.HK', '3300.HK', '1025.HK', '0506.HK', '0906.HK', '0075.HK', '0343.HK', '1004.HK', '8137.HK', '0272.HK', '1808.HK', '3331.HK', '2607.HK', '1000.HK', '2193.HK', '2623.HK', '1280.HK', '0613.HK', '0844.HK', '8379.HK', '1105.HK', '3839.HK', '1681.HK', '1271.HK', '0168.HK', '0256.HK', '0130.HK', '6830.HK', '8237.HK', '0041.HK', '0131.HK', '0859.HK', '2212.HK', '0475.HK', '0722.HK', '0713.HK', '0483.HK', '1360.HK', '1999.HK', '2298.HK', '8005.HK', '0853.HK', '3100.HK', '1113.HK', '0995.HK', '8036.HK', '2196.HK', '0589.HK', '0820.HK', '3091.HK', '0567.HK', '3136.HK', '8219.HK', '0988.HK', '6896.HK', '8305.HK', '0423.HK', '8041.HK', '0351.HK', '3608.HK', '1168.HK', '0116.HK', '0268.HK', '0367.HK', '1002.HK', '1087.HK', '1400.HK', '1138.HK', '0863.HK', '0005.HK', '0809.HK', '2211.HK', '1689.HK', '0229.HK', '0853.HK', '3398.HK', '8172.HK', '6133.HK', '3816.HK', '0399.HK', '3055.HK', '0493.HK', '0132.HK']

stock_name=["AAPL","GOOG","AMZN","TSLA","BABA","PTPI","BTCY",'FOX',"NVDA","DIS","NKE","XOM","KO",
            "2319.HK","0700.HK","1211.HK","6133.hk",'2318.HK','1448.HK','1810.HK',"3690.HK","2018.HK"]
start="2020-12-31"
end="2023-09-30"

from sklearn.decomposition import PCA

#start="2020-12-31"
#end="2022-12-31"

great_moments=[]

#name='6133.hk'
name='1966.HK'

for name in stock_name:
    #MACD619RSI21_8020
    stock_data=pct_change(name,start,end)
    #stock_data=data.get_data_yahoo(tickers=name,start=start,end=end,progress=False)

    #stock_data['Return']=stock_data['Close']/(stock_data['Close'].shift(1))
    stock_data=MACD_short(stock_data)
    stock_data=RSI(stock_data)
    stock_data=RSI_MACD(stock_data,21,80,20)
    stock_data= my_rsi(stock_data, 21)

    stock_data = stock_data.dropna()
    del stock_data["real_situation"]
    #del stock_data["RSI6"]
    #del stock_data["RSI14"]

    #del stock_data["real_situation"]
    #del stock_data["Position"]
    #del stock_data["Close_price_percentage"]

    y = np.where(stock_data['Open'].shift(-1) > stock_data['Open'],1,-1)
    #y = np.where(stock_data['Close'].shift(-1) > stock_data['Close'],1,-1)

    split = int(0.7*len(stock_data))

    pca = PCA(n_components=3)
    sc_X = StandardScaler()

    from sklearn.metrics import accuracy_score
    
    stock_data=stock_data.loc[:,["re_close", "re_open", "my_rsi", "Position", "RSI21"]]
    test_data,train_data=stock_data[split:],stock_data[:split]
    train_data=sc_X.fit_transform(train_data)
    train_data=pca.fit_transform(train_data)
    test_data=sc_X.transform(test_data)
    test_data=pca.transform(test_data)
    y_test,y_train=y[split:],y[:split]

    X_test,X_train=test_data,train_data

    neigh=KNeighborsClassifier(n_neighbors=5)
    neigh.fit(X_train,y_train) 
    y_pred=neigh.predict(X_test)
    res=accuracy_score(y_test,y_pred)

    from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
    #cm = confusion_matrix(y_test, y_pred)
    #ConfusionMatrixDisplay(confusion_matrix=cm).plot()
    accuracy = accuracy_score(y_test, y_pred)
    prec=precision_score(y_test, y_pred)
    rec=recall_score(y_test, y_pred)
    print(name,accuracy,prec,rec)
    #print(accuracy)

AAPL 0.8118811881188119 0.8173913043478261 0.8468468468468469
GOOG 0.8118811881188119 0.8073394495412844 0.8380952380952381
AMZN 0.900990099009901 0.8962264150943396 0.9134615384615384
TSLA 0.8663366336633663 0.8504672897196262 0.8921568627450981
BABA 0.6881188118811881 0.6847826086956522 0.6494845360824743
PTPI 0.7871287128712872 0.7023809523809523 0.7662337662337663
BTCY 0.7673267326732673 0.7407407407407407 0.6976744186046512
FOX 0.8118811881188119 0.7727272727272727 0.8673469387755102
NVDA 0.7821782178217822 0.7850467289719626 0.8
DIS 0.7920792079207921 0.8144329896907216 0.7669902912621359
NKE 0.8118811881188119 0.7745098039215687 0.8404255319148937
XOM 0.8415841584158416 0.8611111111111112 0.8454545454545455
KO 0.8267326732673267 0.8522727272727273 0.7731958762886598


C:\Users\Administrator\Desktop\ARIN7600\ARIN7600-Capstone-main\ARIN7600-Capstone-main\Tool_v1.py:118: RuntimeWarning: invalid value encountered in scalar divide
  res.append(now_raise_sum/now_abs_sum)


2319.HK 0.8214285714285714 0.7802197802197802 0.8255813953488372


C:\Users\Administrator\Desktop\ARIN7600\ARIN7600-Capstone-main\ARIN7600-Capstone-main\Tool_v1.py:118: RuntimeWarning: invalid value encountered in scalar divide
  res.append(now_raise_sum/now_abs_sum)


0700.HK 0.7653061224489796 0.7733333333333333 0.6666666666666666


...

C:\Users\Administrator\Desktop\ARIN7600\ARIN7600-Capstone-main\ARIN7600-Capstone-main\Tool_v1.py:118: RuntimeWarning: invalid value encountered in scalar divide
  res.append(now_raise_sum/now_abs_sum)


1810.HK 0.8010204081632653 0.7777777777777778 0.7865168539325843


C:\Users\Administrator\Desktop\ARIN7600\ARIN7600-Capstone-main\ARIN7600-Capstone-main\Tool_v1.py:118: RuntimeWarning: invalid value encountered in scalar divide
  res.append(now_raise_sum/now_abs_sum)


3690.HK 0.7857142857142857 0.8072289156626506 0.7204301075268817
2018.HK 0.8775510204081632 0.845360824742268 0.9010989010989011


C:\Users\Administrator\Desktop\ARIN7600\ARIN7600-Capstone-main\ARIN7600-Capstone-main\Tool_v1.py:118: RuntimeWarning: invalid value encountered in scalar divide
  res.append(now_raise_sum/now_abs_sum)
